In [ ]:
import numpy as np
import inv_utils as iu
import me_utils as meu
from helita.io import lp
from einops import rearrange
import hmi_plot as hp
from uncertainties import  unumpy
import importlib
import warnings
# Suppress the specific warning
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float32'> type is zero")
warnings.filterwarnings("ignore", message="The value of the smallest subnormal for <class 'numpy.float64'> type is zero")

In [ ]:

importlib.reload(iu)
importlib.reload(meu)
print('reloaded') 

In [ ]:
# Load the configuration from the JSON file
input_config = iu.load_yaml_config('input_config.yaml')

In [ ]:
# Check the input configuration
config = iu.check_input_config(input_config, pprint=True, confirm=False)

In [ ]:

# Extract the input parameters
data_dir = config['data_dir']
save_dir = config['save_dir']
crisp_im = config['crisp_im']
xorg = config['xorg']
xsize = config['xsize']
yorg = config['yorg']
ysize = config['ysize']
xrange = config['xrange']
yrange = config['yrange']
tt = config['time_index']
best_frame_index = config['best_frame_index']
scale = config['scale']
is_north_up = config['is_north_up']
crop = config['crop']
shape = config['shape']
best_frame = config['best_frame']
contrasts = config['contrasts']
hmi_con_series = config['hmi_con_series']
hmi_mag_series = config['hmi_mag_series']
email = config['email']
fov_angle = config['fov_angle']
plot_sst_pointings_flag = config['plot_sst_pointings_flag']
plot_hmi_ic_mag_flag = config['plot_hmi_ic_mag_flag']
plot_crisp_image_flag = config['plot_crisp_image_flag']
verbose = config['verbose']

In [ ]:
# Extract the fits information from the header
fits_info = config['fits_info']
nx = fits_info['nx']
ny = fits_info['ny']
mu = fits_info['mu']
x1 = fits_info['hplnt'][tt][0]
x2 = fits_info['hplnt'][tt][1]
y1 = fits_info['hpltt'][tt][0]
y2 = fits_info['hpltt'][tt][1]
tobs = fits_info['all_start_times'][tt]
tstart = fits_info['start_time_obs']
tend = fits_info['end_time_obs']
hplnt = fits_info['hplnt']
hpltt = fits_info['hpltt']
all_start_times = fits_info['all_start_times']
central_wavelength = fits_info['central_wavelength']

In [ ]:
# Reset the x and y ranges if cropping is enabled
if crop:
    x_list = np.linspace(x1, x2, num=nx)
    y_list = np.linspace(y1, y2, num=ny)
    x_list = x_list[xrange[0]:xrange[1]]
    y_list = y_list[yrange[0]:yrange[1]]
    x1 = x_list[0]
    x2 = x_list[-1]
    y1 = y_list[0]
    y2 = y_list[-1]
    nx = xsize
    ny = ysize

In [ ]:
if plot_sst_pointings_flag:
    hp.plot_sst_pointings(tstart, hmi_con_series, hplnt, hpltt,figsize=(6, 6), email=email, save_dir=save_dir)

In [ ]:
if plot_hmi_ic_mag_flag:
    hp.plot_hmi_ic_mag(tobs, hmi_con_series, hmi_mag_series, email, x1, x2, y1, y2, save_dir=save_dir, figsize=(10, 5),  is_north_up=is_north_up, fov_angle=fov_angle, shape=shape)

In [ ]:
if plot_crisp_image_flag:
    print('SST CRISP image with North up:', not(is_north_up))
    iu.plot_crisp_image(crisp_im, tt=tt, ss=0, ww=0, figsize=(6,6), fontsize=10, rot_fov=fov_angle, north_up=not(is_north_up), crop=crop, xrange=xrange, yrange=yrange, xtick_range=[x1,x2], ytick_range=[y1,y2])

In [ ]:
inversion_config = iu.load_yaml_config('inversion_config.yaml')
# Load the variables from the inversion configuration
dtype = inversion_config['dtype']
nthreads = inversion_config['nthreads']
sigma_strength= inversion_config['sigma_strength']
sigma_list = inversion_config['sigma_list']
erh = inversion_config['erh']
init_model_params = inversion_config['init_model_params']
nRandom1 = inversion_config['nRandom1']
nIter1 = inversion_config['nIter1']
chi2_thres1 = inversion_config['chi2_thres1']
median_filter_chi2_mean_thres = inversion_config['median_filter_chi2_mean_thres']
median_filter_size = inversion_config['median_filter_size']
nRandom2 = inversion_config['nRandom2']
nIter2 = inversion_config['nIter2']
chi2_thres2 = inversion_config['chi2_thres2']
nIter3 = inversion_config['nIter3']
chi2_thres3 = inversion_config['chi2_thres3']
alpha_strength = inversion_config['alpha_strength']
alpha_list = inversion_config['alpha_list']
nan_mask_replacements = inversion_config['nan_mask_replacements']


In [ ]:
ll = meu.load_crisp_frame(crisp_im, tt, crop=crop, xrange=xrange, yrange=yrange)

In [ ]:
obs, sig, l0, me = meu.init_me_model(ll, sigma_strength, sigma_list, erh=erh, dtype=dtype, nthreads=nthreads)

In [ ]:
Imodel = meu.init_model(me, ny, nx, init_model_params=init_model_params, dtype=dtype)

In [ ]:
Imodel, syn, chi2 = meu.run_randomised_me_inversion(Imodel, me, obs, sig, nRandom=nRandom1, nIter=nIter1, chi2_thres=chi2_thres1, mu=mu, verbose=verbose)
masked_chi2_mean = iu.masked_mean(chi2, ll.mask)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(Imodel, ll.mask, scale=scale, save_fig=False)
    iu.plot_mag(Imodel, ll.mask, scale=scale, save_fig=False)

In [ ]:
Imodel = meu.apply_median_filter_based_on_chi2(Imodel, masked_chi2_mean, median_filter_chi2_mean_thres, median_filter_size)
if verbose:    
    iu.plot_inversion_output(Imodel,ll.mask,scale=scale, save_fig=False)
    iu.plot_mag(Imodel,ll.mask,scale=scale, save_fig=False)

In [ ]:
Imodel, syn, chi2 = meu.run_randomised_me_inversion(Imodel, me, obs, sig, nRandom=nRandom2, nIter=nIter2, chi2_thres=chi2_thres2, mu=mu, verbose=verbose)
masked_chi2_mean = iu.masked_mean(chi2, ll.mask)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(Imodel, ll.mask, scale=scale, save_fig=False)
    iu.plot_mag(Imodel, ll.mask, scale=scale, save_fig=False)

In [ ]:
mo, syn, chi2 = meu.run_spatially_regularized_inversion(me, Imodel, obs, sig, nIter3, chi2_thres3, mu, alpha_strength, alpha_list, method=1, delay_bracket=3, dtype=dtype,verbose=True)
errors = me.estimate_uncertainties(np.squeeze(mo), obs, sig, mu=mu)
corrected_mo = meu.correct_velocities_for_cavity_error(mo, ll.cmap, l0, global_offset=0.0)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(corrected_mo,ll.mask,scale=scale, save_fig=False)
    iu.plot_mag(corrected_mo,ll.mask,scale=scale, save_fig=False)

In [ ]:
masked_model= meu.apply_mask_to_model(corrected_mo, ll.mask, nan_mask_replacements)
masked_errors = meu.apply_mask_to_model(errors, ll.mask, nan_mask_replacements)
if verbose:
    print(f'Masked chi2 mean: {masked_chi2_mean:.2f}')
    iu.plot_inversion_output(masked_model,mask=None,scale=scale, save_fig=False)
    iu.plot_inversion_output(masked_errors,mask=None,scale=scale, save_fig=False, apply_median_filter=True, filter_index=[1,2], filter_size=3)
    iu.plot_mag(masked_model,mask=None,scale=scale, save_fig=False)

In [ ]:
# Save masked model and errors using numpy
np.save(save_dir + 'masked_model.npy', masked_model)
np.save(save_dir + 'masked_errors.npy', masked_errors)

---

In [ ]:
# load masked_model and masked_errors from save_dir using numpy
# Load the saved masked model and errors
masked_model = np.load(save_dir + 'masked_model.npy')
masked_errors = np.load(save_dir + 'masked_errors.npy')

# Verify the loaded arrays
print("Masked Model:")
print(masked_model.shape)

print("\nMasked Errors:")
print(masked_errors.shape)

In [ ]:
# iu.plot_sst_blos_bhor(blos_cube, bhor_cube, tt=tt,xrange=xrange, yrange=yrange, figsize=(20,10), fontsize=12, crop=crop)

In [ ]:
importlib.reload(iu)
importlib.reload(meu)

In [ ]:

model_im = rearrange(masked_model, 'ny nx nparams -> nparams ny nx')
errors_im = rearrange(masked_errors, 'ny nx nparams -> nparams ny nx')

In [ ]:
# Create arrays with uncertainties
B_with_errors = unumpy.uarray(model_im[0], errors_im[0])
inc_with_errors = unumpy.uarray(model_im[1], errors_im[1])

# Calculate Blos and Bhor with propagated errors
Blos_with_errors = B_with_errors * unumpy.cos(inc_with_errors)
Bhor_with_errors = B_with_errors * unumpy.sin(inc_with_errors)

# Extract nominal values and standard deviations
Blos = unumpy.nominal_values(Blos_with_errors)
Blos_err = unumpy.std_devs(Blos_with_errors)

Bhor = unumpy.nominal_values(Bhor_with_errors)
Bhor_err = unumpy.std_devs(Bhor_with_errors)


In [ ]:
Bhor_err_clipped = np.clip(Bhor_err, a_min=0, a_max=np.max(Bhor))
Blos_err_clipped = np.clip(Blos_err, a_min=0, a_max=np.max(np.abs(Blos)))

In [ ]:

# Extend model_im by adding Blos and Bhor and 10 and 11 indices
model_im = np.concatenate((model_im, Blos[np.newaxis], Bhor[np.newaxis]), axis=0)
errors_im = np.concatenate((errors_im, Blos_err_clipped[np.newaxis], Bhor_err_clipped[np.newaxis]), axis=0)

In [ ]:

idl_model_im = rearrange(model_im, 'nparams ny nx -> nparams nx ny')
idl_errors_im = rearrange(errors_im, 'nparams ny nx -> nparams nx ny')

In [ ]:
idl_model_im.shape

In [ ]:
# save the idl_model_im and idl_errors_im using numpy
np.save(save_dir + 'idl_model_im.npy', idl_model_im)
np.save(save_dir + 'idl_errors_im.npy', idl_errors_im)

In [ ]:
# load the idl_model_im and idl_errors_im using numpy
idl_model_im = np.load(save_dir + 'idl_model_im.npy')
idl_errors_im = np.load(save_dir + 'idl_errors_im.npy')

print(f"IDL Model image: {idl_model_im.shape}")
print(f"IDL Errors image: {idl_errors_im.shape}")

In [ ]:
# combine input_config and inversion_config dictionaries
full_config = {**input_config, **inversion_config}

In [ ]:
fits_header = iu.load_fits_header(crisp_im, out_dict=False)

In [ ]:
inversion_out_list = ["Bstr", "Binc", "Bazi", "Vlos", "Vdop", "etal", "damp", "S0", "S1", "Blos", "Bhor"]
inverstion_error_out_list = ["Bstr_err", "Binc_err", "Bazi_err", "Vlos_err", "Vdop_err", "etal_err", "damp_err", "S0_err", "S1_err", "Blos_err", "Bhor_err"]

In [ ]:
inversion_save_fits_list = ["Bstr", "Binc", "Bazi", "Vlos", "Blos", "Bhor"]
inversion_save_errors_fits = True 

In [ ]:
time_string = all_start_times[tt].replace(':', '').replace(' ', '_T')


In [ ]:
for var in inversion_save_fits_list:
    var_index = inversion_out_list.index(var)
    out_file_name = save_dir + f'temp_{inversion_out_list[var_index]}_{str(int(central_wavelength))}_{time_string}.fits'
    iu.save_fits(idl_model_im[var_index], fits_header, out_file_name, overwrite=True)
    if inversion_save_errors_fits:
        out_file_name = save_dir + f'temp_{inversion_out_list[var_index]}_err_{str(int(central_wavelength))}_{time_string}.fits'
        iu.save_fits(idl_errors_im[var_index], fits_header, out_file_name, overwrite=True)

In [ ]:
bazi_im = idl_model_im[2]
blos_im = idl_model_im[-2]
bhor_im = idl_model_im[-1]
blos_im.shape

In [ ]:
inversion_save_lp_list = ["Blos", "Bhor", "Bazi", "Vlos"]
inversion_save_errors_lp = True

In [ ]:
for var in inversion_save_lp_list:
    var_index = inversion_out_list.index(var)
    lp_out_file_name = save_dir + f'temp_{inversion_out_list[var_index]}_{str(int(central_wavelength))}_{time_string}.fcube'
    lp_data = np.float32(idl_model_im[var_index])
    if len(lp_data.shape) == 2:
        lp_data = np.reshape(lp_data,(nx, ny,1))
    lp.writeto(lp_out_file_name, lp_data, extraheader='', dtype=None, verbose=True, append=False)
    if inversion_save_errors_lp:
        lp_out_file_name = save_dir + f'temp_{inversion_out_list[var_index]}_err_{str(int(central_wavelength))}_{time_string}.fcube'
        lp_data = np.float32(idl_errors_im[var_index])
        if len(lp_data.shape) == 2:
            lp_data = np.reshape(lp_data,(nx, ny,1))
        lp.writeto(lp_out_file_name, lp_data, extraheader='', dtype=None, verbose=True, append=False)
    

In [ ]:
ff = iu.load_fits_data(out_file_name)

In [ ]:
iu.plot_image(ff)

In [ ]:
hh = iu.load_fits_header('temp/inv_mos.fits')

#### Things to complete
- [x] Move all the inputs to a dictionary and later save them in the header of the output file. Also add the best seeing frame number.
- [x] Move the preprocessing steps like plotting and FOV details as an optional but default true step
- [x] Plot a rectangle to show cropping region is true
- [x] Save fits with [blos, theta, phi, vlos + errors + mask] for each frame (temporarily) and later combine for final fits
- [x] Check for option to convert to fcube and icube formats using ispy or helita tools
- [ ] Add option to do only one frame separately if user wants.
- [ ] Add fov angle and other inputs needed for ambiguity resolution and remap in header

#### To do for final cube
- [x] Pick the best seeing frame from the dataset
- [x] Run the full inversion for the best seeing frame
- [ ] Use this output as an initial guess for the other frames
